# Condition monitoring of hydraulic system

<Use Case library>

|  |  |
|--|--|
|**Title:** | Condition monitoring of hydraulic system |
|**Author:** | Michal Bezak, Tangent Works|
|**Industry:** | Manufacturing, Agriculture, Transportation |
|**Area:** | Predictive maintenance, Condition monitoring |
|**Type:** | Forecasting |


## Description

Hydraulic system on test rig consists of a primary working and a secondary cooling-filtration circuit, both connected via the oil tank. The system operates in 60 seconds cycles. From operations perspective it is desired to understand when system will be in sub-optimal condition to avoid malfunction or even outage.

Proactive condition monitoring can improve efficiency and maximized uptime of machine. Equally, it can be part of predictive maintenance process which covers variety of areas, including prediction of failure.

To avoid downtimes caused by malfunction of technical equipment, companies follow maintenance schedule to discover and fix potential issues. Nevertheless, with more data (from sensors installed) available there are new possibilities how to make maintenance process better. With AI/ML it is possible to further optimize maintenance schedule.

This has tangible financial implications, imagine two extreme situations:

- Maintenance planned for later than needed - tech. equipment would reach point of failure and cause operations disruptions, every minute of downtime can bring big losses, delays of other parts of the process, not to mention implications to health or lives of people.
- If a company makes maintenance too often or too soon, expenses for time and material spent is higher than situation required, also, should component be replaced completely, capital invested into machine/components is not utilized to its full potential.

Hydraulic systems are commonly used in heavy equipment. In a hydraulic system, pressure, applied to a contained fluid is transmitted. That pressurized fluid acts upon every part of the section of a containing vessel and creates force or power. Due to the use of this force, it is possible for instance to lift heavy loads. Transporting liquid through a set of interconnected components, a hydraulic circuit is a system that can control where fluid flows as well as control fluid pressure. Thus, it consists of multiple components and offers range of areas to monitor to ensure optimal operation. In our use case we will focus on pump leakage.

In this Use case we will solve following problem: *How much time operating in optimal condition remains (until it reaches sub-optimal condition)?*. From ML perspective this can be framed as forecasting problem. 


## Business parameters

|  |  |
|--|--|
|**Business objective:** | Reduce outages in production process |
|**Business value:** | Reduce cost of down-times and inefficient operations |
|**KPI:** | - |

|  |  |
|--|--|
|**Business objective:** | Predictive maintenance |
|**Business value:** | Optimal cost |
|**KPI:** | - |

## Dataset

The dataset was obtained with a hydraulic test rig. The system cyclically repeats constant load cycles (duration of 60 sec.) and measures values such as: pressure, volume flow and temperature. Meanwhile the condition of hydraulic components - cooler, valve, pump, and accumulator - is quantified. 

One of conditions monitored - *internal pump leakage* - is selected and transformed into target variable. In dataset it was named *rul* which stands for remaining useful life although its complete description would fit remaining time of optimal operating condition.


### Sampling

Raw data files were transformed into time series dataset with 1-second sampling rate. Raw data were sampled at variable frequency, e.g. pressure 100x per second, volume flow at 10 Hz, so aggregation was necessary to synchronize predictors in dataset. *median* was used for aggregation.

### Data

Structure of CSV file:



RUL values were encoded based on time remaining until the condition of *internal pump leakage* degraded to level 2, i.e. severe leakage. (Condition values in raw dataset are represented as follows: 0 for no leakage, 1 for weak leakage, 2 for severe leakage).

### Data situation

If we want TIM to quantify current condition based on measurement, it means that we want to predict value of target based on predictors values, and so the last record of target must be kept empty (NaN/None) in dataset. TIM will use available predictors to predict given record. This situation will be replicated by TIM to calculate results for all out-of-sample records.

CSV files used in experiments can be downloaded [here](data_pump_leakage.csv).

### Source

Raw data files were acquired at [Kaggle](https://www.kaggle.com/mayank1897/condition-monitoring-of-hydraulic-systems).

|  |  |  |  |
|--|--|--|--|
| Column name | Description |Type | Availability|
| **Timestamp** | Date| Timestamp column | |
| **rul** | Remaning useful life (no. of cycles) | Target | t-1 |
| **cycle** | Operations cycle no. |Predictor | t+1 |
| **TS1 ... TS4** |  Temperature (Celsius)  | Predictor | t+1 |
| **PS1 ... PS6** |   Pressure (bars)  | Predictor | t+1 |
| **VS1** | Vibration (mm/s) | Predictor | t+1 |
| **FS1 ... FS2** | Volume flow (l/min) | Predictor | t+1 |
| **CP** | Cooling power (virtual, kW)  | Predictor | t+1 |
| **CE** | Cooling efficiency (virtual, %)| Predictor | t+1 |
| **SE** | Efficiency factor (%) | Predictor | t+1 |
| **EPS1** | Motor power (W) | Predictor | t+1 |

# 1. Setup

In [1]:
import json
import logging
import tim_client
import pandas as pd
import datetime as dt
import plotly as plt
import plotly.express as px
import plotly.graph_objects as go

# import requests
# import numpy as np
# from sklearn.metrics import mean_squared_error, mean_absolute_error
# import math

credentials_json = json.load(open('credentials.json'))
logging.basicConfig(level= logging.getLevelName('INFO'), format='[%(levelname)s] %(asctime)s - %(name)s:%(funcName)s:%(lineno)s - %(message)s')
logger = logging.getLogger(__name__)
credentials = tim_client.Credentials(credentials_json['license_key'], credentials_json['email'], credentials_json['password'], tim_url='https://timws.tangent.works/latest/api')
api_client = tim_client.ApiClient(credentials)
api_client.save_json = True
api_client.json_saving_folder_path = 'logs/'

[INFO] 2021-06-09 10:23:23,945 - tim_client.api_client:save_json:66 - Saving JSONs functionality has been enabled
[INFO] 2021-06-09 10:23:23,947 - tim_client.api_client:json_saving_folder_path:75 - JSON destination folder changed to logs


# 2. Data Preprocessing

In [1]:
data = tim_client.load_dataset_from_csv_file('data_pump_leakage.csv', sep=',')
data

NameError: name 'tim_client' is not defined

In [2]:
timestamp = 'time'
target_variable = 'rul'
training_testing_split = 0.3
backtest_length = int(data.shape[0]*training_testing_split )

NameError: name 'data' is not defined

In [3]:
nb_rows = len(data.columns)
p_list = list(data.columns[1:nb_rows+1])
x_axis = timestamp
fig = plt.subplots.make_subplots(rows=nb_rows-1, cols=1, shared_xaxes=True, vertical_spacing=0.02)
for p in p_list:
    fig.add_trace(go.Scatter(x=data[x_axis], y=data[p], name=p), row=p_list.index(p)+1, col=1)
fig.update_layout(height=2500, width=1200, title_text="Data visualization")
fig.show()    

NameError: name 'data' is not defined

### Visualization of RUL value until failure points

In [4]:
vis_df = data.copy()
vis_df['time'] = pd.to_datetime( vis_df['time'] )
vis_df.set_index('time',inplace=True)
timestamps = [ vis_df.index.min() + i * dt.timedelta(seconds=1) for i in range( int( ( vis_df.index.max() - vis_df.index.min() ).total_seconds() ) + 1 ) ]
temp_df = pd.DataFrame( {'timestamp': timestamps } )
temp_df.set_index( 'timestamp', inplace=True )
vis_df = temp_df.join( vis_df )
vis_df['time'] = vis_df.index

NameError: name 'data' is not defined

In [5]:
fig = go.Figure(go.Scatter(x=vis_df[timestamp],y=vis_df[target_variable]/vis_df[target_variable].max(),name=target_variable))
fig.update_layout(title = target_variable,height = 700,width = 1200)
fig.show()

NameError: name 'go' is not defined

# 3. Engine settings

Parameters that need to be set are:

- Prediction horizon (Sample + 1) - because we want to predict the next value only.
- Back-test length (that defines out-of-sample interval).
- setting *modelQuality* to *Medium* would prevent TIM to use lagged values of target itself which is expected otherwise it would basically leak unknown information; setting *allowOffsets* to False will switch off use of lagged values completely including predictors - bottom line is to one of these settings. Results of experiments did not deliver substantially different results using one or the other.

We also ask for additional data from engine to see details of sub-models so we define *extendedOutputConfiguration* parameter as well.

In [6]:
rtiml_configuration = {
    'usage': {                                 
        'predictionTo': {'baseUnit': 'Sample','offset': 1}, 
        'backtestLength': backtest_length,
        'modelQuality': [ { 'day': 0, 'quality': 'Medium' } ]
    },    
    #'allowOffsets': False,
    'extendedOutputConfiguration': {
        'returnExtendedImportances': True,
    }
}

NameError: name 'backtest_length' is not defined

# 4. Run TIM

In [7]:
uuids = api_client.prediction_build_model_predict(data,rtiml_configuration,wait_to_finish = False).request_uuid
while True:
    rtiml_prediction = api_client.prediction_build_model_predict_detail(uuids)
    print(rtiml_prediction.status+" "+str(rtiml_prediction.progress),end='\r')
    if rtiml_prediction.status == "Running": continue 
    else: break

print(rtiml_prediction.status+" "+str(rtiml_prediction.progress))
for i in rtiml_prediction.events:
    if "Warning" in i['message']: print(i['message'])
print('Runtime: '+str(pd.to_datetime(pd.DataFrame(rtiml_prediction.events)['dateTime'].iloc[-1])-(pd.to_datetime(pd.DataFrame(rtiml_prediction.events)['dateTime']).iloc[0])))

NameError: name 'api_client' is not defined

# 5. Collect Results

In [8]:
tim_input = data
# ------------------------------ Predictions ------------------------------
prediction_df = rtiml_prediction.get_prediction(include_intervals=True).reset_index()
prediction_df['Timestamp'] = pd.to_datetime(prediction_df['Timestamp']).dt.strftime('%Y-%m-%d %H:%M:%S')
# ------------------------------ Aggregated Predictions & Accuracy ------------------------------
aggregated_predictions_df = pd.DataFrame()
accuracy_metrics = pd.DataFrame()
for ag in rtiml_prediction.aggregated_predictions:
    ag_df = pd.DataFrame(ag['values']).reset_index()
    ag_df['day'] = ag['day']
    ag_df = ag_df.rename(columns={'Prediction':ag['type']})
    aggregated_predictions_df = aggregated_predictions_df.append(ag_df)
    acc_df = pd.json_normalize(ag['accuracyMetrics'])
    acc_df['day'] = ag['day']
    acc_df['type'] = ag['type']
    accuracy_metrics = accuracy_metrics.append(acc_df)
aggregated_predictions_df['Timestamp'] = pd.to_datetime(aggregated_predictions_df['Timestamp']).dt.strftime('%Y-%m-%d %H:%M:%S')
# ------------------------------ Insights ------------------------------
pi_df = pd.DataFrame(rtiml_prediction.predictors_importances['simpleImportances'])
fi_df = pd.DataFrame(rtiml_prediction.predictors_importances['extendedImportances'])
# ------------------------------ Events ------------------------------
events_df = pd.DataFrame(rtiml_prediction.events)
# ------------------------------ Actuals ------------------------------
actuals_df = tim_input[[timestamp,target_variable]].rename(columns={timestamp:'Timestamp',target_variable:'Actuals'}).dropna()
actuals_df['Timestamp'] = pd.to_datetime(actuals_df['Timestamp']).dt.strftime('%Y-%m-%d %H:%M:%S')
# ------------------------------ Results ------------------------------
results_df = actuals_df.append(prediction_df).merge(aggregated_predictions_df[['Timestamp','inSample','outOfSample']],on='Timestamp',how='left')

NameError: name 'data' is not defined

In [ ]:
out_of_sample_predictions = rtiml_prediction.aggregated_predictions[1]['values']
out_of_sample_predictions.rename( columns = {'Prediction':target_variable+'_pred'}, inplace=True)
out_of_sample_timestamps = out_of_sample_predictions.index.tolist()
evaluation_data = data.copy()
evaluation_data[timestamp] = pd.to_datetime(data[timestamp]).dt.tz_localize('UTC')
evaluation_data = evaluation_data[ evaluation_data[ timestamp ].isin( out_of_sample_timestamps ) ]
evaluation_data.set_index( timestamp,inplace=True)
evaluation_data = evaluation_data[[target_variable]].join(out_of_sample_predictions)
temp_df = pd.DataFrame( {'timestamp': [ evaluation_data.index[0] + i * dt.timedelta(seconds=1) for i in range( int( (evaluation_data.index[-1] - evaluation_data.index[0]).total_seconds() ) + 1 ) ] } )
temp_df.set_index( 'timestamp', inplace=True )
evaluation_data = temp_df.join( evaluation_data )

# 6. Visualize Results

In [9]:
v_data = results_df
x_axis = 'Timestamp'
actuals = 'Actuals'
prediction = 'Prediction'
inSample = 'inSample'
outOfSample = 'outOfSample'
upper = 'UpperValues'
lower = 'LowerValues'

fig = go.Figure(go.Scatter(x=actuals_df[x_axis], y=actuals_df[actuals], name=actuals, line=dict(color='black')))
fig.add_trace(go.Scatter(x=prediction_df[x_axis], y=prediction_df[prediction], name=prediction, line=dict(color='goldenrod')))
fig.add_trace(go.Scatter(x=prediction_df[x_axis], y=prediction_df[upper], name=upper, line=dict(color='rgba(240,140,0,0.3)')))
fig.add_trace(go.Scatter(x=prediction_df[x_axis], y=prediction_df[lower], name=lower, line=dict(color='rgba(240,140,0,0.3)')))
fig.add_trace(go.Scatter(x=aggregated_predictions_df[x_axis], y=aggregated_predictions_df[inSample], name=inSample, line=dict(color='green')))
fig.add_trace(go.Scatter(x=aggregated_predictions_df[x_axis], y=aggregated_predictions_df[outOfSample], name=outOfSample, line=dict(color='red')))
fig.update_layout(height=500, width=1200, title_text="Results")
fig.show()

NameError: name 'results_df' is not defined

In [11]:
print('Predictors not used:'+str(list(set(tim_input.columns[1:])-set(pi_df['predictorName']))))
x_axis = 'predictorName'
y_axis = 'importance'
fig1 = go.Figure(go.Bar(x=pi_df[x_axis], y=pi_df[y_axis]))
fig1.update_layout(height=700,width=1400,title_text='Predictor Importances',xaxis_title=x_axis,yaxis_title=y_axis)
fig1.show()

NameError: name 'tim_input' is not defined

In [12]:
fig = px.sunburst(fi_df, path=['time','termName'], values='importance',color='termName')
fig.update_layout(height=700,width=700,title_text='Feature Importances')
fig.show()

NameError: name 'px' is not defined

In [15]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=evaluation_data.index,y=evaluation_data[target_variable+'_pred'],name='Predicted'))
fig.add_trace(go.Scatter(x=evaluation_data.index,y=evaluation_data[target_variable],name='Actual'))
fig.update_layout(title = 'Ouf of sample results',height = 700,width = 1200)
fig.show()

NameError: name 'go' is not defined

# 7. Save Results

In [17]:
results_df.to_csv('results_df.csv',index=False)
tim_input.to_csv('tim_input.csv',index=False)
evaluation_data.to_csv('evaluation_data.csv',index=False)
pi_df.to_csv('pi_df.csv',index=False)
fi_df.to_csv('fi_df.csv',index=False)
events_df.to_csv('events_df.csv',index=False)

NameError: name 'results_df' is not defined

### Summary

Chart above depicts how TIM quantified RUL for three out-of-sample occasions.

We can see that as it gets close to points of "failure", it can rise and reverse its trend; this may need to be explored further, it is expected to be linked to operational specifics of given sub-system also, there are various options how to address this, for instance address sudden reversion, or evaluate value only above certain threshold.

Condition monitoring is vastly linked to deeper understanding of given subsystem which is visible also in our use case. Heuristics employed in encoding condition levels plays vital role in preparing data for model building and use in operations.

In this use case, we demonstrated how TIM can support estimate of time remaining in optimal condition and/or before component fails, almost in real time operations set-up. Notice that actual cycle no. is not included among the list of predictors, TIM managed to build powerful prediction model without it.
